https://github.com/dbouchabou/Fully-Convolutional-Network-Smart-Homes/tree/master

In [ ]:
from google.colab import drive
drive.mount('/content/drive' , force_remount = True)

Mounted at /content/drive


In [ ]:
import os
import numpy as np
import pandas as pd
import pickle
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
pip install tensorflow

### MHEALTH

In [ ]:
cd "/content/drive/MyDrive/MHEALTHDATASET/"

/content/drive/MyDrive/MHEALTHDATASET


In [ ]:
df_mhealth = pd.read_pickle('mhealth_for_lstm.pkl')

#### sliding windows

In [ ]:
def segment_activities(df):
    activitiesSeq = []

    ponentialIndex = df.activity.ne(df.activity.shift())

    ii = np.where(ponentialIndex == True)[0]

    for i,end in enumerate(ii):
        if i > 0 :

          dftmp = df[ii[i-1]:end]
          activitiesSeq.append(dftmp)
    return activitiesSeq

In [ ]:
activitySequences = segment_activities(df_mhealth)

In [ ]:
def generate_sentence(df2):
    sentence = ""

    #columns for which to retain values
    columns_of_interest = ['acc_ch_x', 'acc_ch_y', 'acc_ch_z', 'ecg_sig_1', 'ecg_sig_2',
       'acc_la_x', 'acc_la_y', 'acc_la_z', 'gyr_la_x', 'gyr_la_y', 'gyr_la_z',
       'mag_la_x', 'mag_la_y', 'mag_la_z', 'acc_rw_x', 'acc_rw_y', 'acc_rw_z',
       'gyr_rw_x', 'gyr_rw_y', 'gyr_rw_z', 'mag_rw_x', 'mag_rw_y', 'mag_rw_z',
       'activity', 'subject']

    #iterate over columns
    for column in columns_of_interest:
        #get col value
        value = df2[column].values[0]

        #add col name and value
        sentence += "{}{}".format(column, value)

        #space for all columns except last
        if column != columns_of_interest[-1]:
            sentence += " "

    return sentence

In [ ]:
def sequencesToSentences(activitySequences):
	sentences = []
	label_sentences = []

	for i in range(len(activitySequences)):

		sentence = generate_sentence(activitySequences[i])

		sentences.append(sentence)
		label_sentences.append(activitySequences[i].activity.values[0])

	return sentences, label_sentences

In [ ]:
sentences, label_sentences = sequencesToSentences(activitySequences)

tokenizer and indexing

In [ ]:
tokenizer = Tokenizer(filters='!"#$%&()*+,-/:;<=>?@[\\]^_`{|}~\t\n')
tokenizer.fit_on_texts(sentences)
word_index = tokenizer.word_index
indexed_sentences = tokenizer.texts_to_sequences(sentences)

#### sliding window data segmentation

In [ ]:
def slidingWindow(sequence,winSize,step=1):

    try: it = iter(sequence)
    except TypeError:
        raise Exception("**ERROR** sequence must be iterable.")
    if not ((type(winSize) == type(0)) and (type(step) == type(0))):
        raise Exception("**ERROR** type(winSize) and type(step) must be int.")
    if step > winSize:
        raise Exception("**ERROR** step must not be larger than winSize.")

    numOfChunks = int(((len(sequence)-winSize)/step)+1)

    # Do the work
    if winSize > len(sequence):
        yield sequence[0:len(sequence)]
    else:
        for i in range(0,numOfChunks*step,step):
            yield sequence[i:i+winSize]

In [ ]:
X_windowed = []
Y_windowed = []
winSize = 50
step = 1

for i,s in enumerate(indexed_sentences):
	chunks = slidingWindow(s,winSize,step)
	for chunk in chunks:
		X_windowed.append(chunk)
		Y_windowed.append(label_sentences[i])

In [ ]:
padded_x = pad_sequences(X_windowed)
padded_y = np.array(Y_windowed)

In [ ]:
from scipy.stats import entropy

js_divergences = np.zeros(x_train.shape[1])
for i in range(x_train.shape[1]):
    # Compute histograms with the same bins for both X_train and X_test
    bins = max(len(np.unique(x_train[:, i])), len(np.unique(x_test[:, i])))
    p, _ = np.histogram(x_train[:, i], bins=bins, density=True)
    q, _ = np.histogram(x_test[:, i], bins=bins, density=True)
    m = 0.5 * (p + q)
    js_divergences[i] = 0.5 * (entropy(p, m) + entropy(q, m))

avg_js_divergence = np.mean(js_divergences)
print("Average Jensen-Shannon divergence:", avg_js_divergence)

Average Jensen-Shannon divergence: 0.12609317940121165


### PAMAP2

In [ ]:
cd "/content/drive/MyDrive/PAMAP2_Dataset"

/content/drive/MyDrive/PAMAP2_Dataset


In [ ]:
data = pd.read_pickle('pamap2_for_lstm.pkl')

scaler

In [ ]:
from sklearn.preprocessing import StandardScaler
activity_id = data['activity_id']
sub_id = data['id']

df_without_activity_id = data.drop(columns=['activity_id', 'id'])
scaler = StandardScaler()
data_scaled = scaler.fit_transform(df_without_activity_id)
data = pd.DataFrame(data_scaled, columns=df_without_activity_id.columns, index=df_without_activity_id.index)
data = pd.concat([activity_id, sub_id, data], axis=1)

In [ ]:
def segment_activities(df):
    activitiesSeq = []

    ponentialIndex = df.activity_id.ne(df.activity_id.shift())

    ii = np.where(ponentialIndex == True)[0]

    for i,end in enumerate(ii):
        if i > 0 :

          dftmp = df[ii[i-1]:end]
          activitiesSeq.append(dftmp)
    return activitiesSeq

activitySequences = segment_activities(data)

#### with my columns

In [ ]:
def generate_sentence(df2):
    sentence = ""

    #columns for which to retain values
    columns_of_interest = ['heart_rate', 'hand_temperature', 'hand_3D_acceleration_16_x', 'hand_3D_acceleration_16_y',
                           'hand_3D_acceleration_16_z', 'hand_3D_acceleration_6_x', 'hand_3D_acceleration_6_y',
                           'hand_3D_acceleration_6_z', 'hand_3D_gyroscope_x', 'hand_3D_gyroscope_y',
                           'hand_3D_gyroscope_z', 'hand_3D_magnetometer_x', 'hand_3D_magnetometer_y',
                           'hand_3D_magnetometer_z', 'chest_temperature', 'chest_3D_acceleration_16_x',
                           'chest_3D_acceleration_16_y', 'chest_3D_acceleration_16_z', 'chest_3D_acceleration_6_x',
                           'chest_3D_acceleration_6_y', 'chest_3D_acceleration_6_z', 'chest_3D_gyroscope_x',
                           'chest_3D_gyroscope_y', 'chest_3D_gyroscope_z', 'chest_3D_magnetometer_x',
                           'chest_3D_magnetometer_y', 'chest_3D_magnetometer_z', 'ankle_temperature',
                           'ankle_3D_acceleration_16_x', 'ankle_3D_acceleration_16_y', 'ankle_3D_acceleration_16_z',
                           'ankle_3D_acceleration_6_x', 'ankle_3D_acceleration_6_y', 'ankle_3D_acceleration_6_z',
                           'ankle_3D_gyroscope_x', 'ankle_3D_gyroscope_y', 'ankle_3D_gyroscope_z',
                           'ankle_3D_magnetometer_x', 'ankle_3D_magnetometer_y', 'ankle_3D_magnetometer_z']

    #iterate over columns
    for column in columns_of_interest:
        value = df2[column].values[0]  # Directly access the single value in the column

        sentence += "{}{}".format(column, value)

        if column != columns_of_interest[-1]:
            sentence += " "

    return sentence

In [ ]:
def sequencesToSentences(activitySequences):
	sentences = []
	label_sentences = []

	for i in range(len(activitySequences)):

		sentence = generate_sentence(activitySequences[i])

		sentences.append(sentence)
		label_sentences.append(activitySequences[i].activity_id.values[0])

	return sentences, label_sentences

sentences, label_sentences = sequencesToSentences(activitySequences)

sentences indexization

In [ ]:
tokenizer = Tokenizer(filters='!"#$%&()*+,-/:;<=>?@[\\]^_`{|}~\t\n')
tokenizer.fit_on_texts(sentences)
word_index = tokenizer.word_index
indexed_sentences = tokenizer.texts_to_sequences(sentences)

sliding windows

In [ ]:
def slidingWindow(sequence,winSize,step=1):

    try: it = iter(sequence)
    except TypeError:
        raise Exception("**ERROR** sequence must be iterable.")
    if not ((type(winSize) == type(0)) and (type(step) == type(0))):
        raise Exception("**ERROR** type(winSize) and type(step) must be int.")
    if step > winSize:
        raise Exception("**ERROR** step must not be larger than winSize.")

    numOfChunks = int(((len(sequence)-winSize)/step)+1)

    # Do the work
    if winSize > len(sequence):
        yield sequence[0:len(sequence)]
    else:
        for i in range(0,numOfChunks*step,step):
            yield sequence[i:i+winSize]

In [ ]:
X_windowed = []
Y_windowed = []
winSize = 50
step = 1

for i,s in enumerate(indexed_sentences):
	chunks = slidingWindow(s,winSize,step)
	for chunk in chunks:
		X_windowed.append(chunk)
		Y_windowed.append(label_sentences[i])

In [ ]:
padded_x = pad_sequences(X_windowed)
padded_y = np.array(Y_windowed)

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(padded_x, padded_y, test_size=(1 - 0.6), random_state=42)
x_test, x_val, y_test, y_val = train_test_split(x_test, y_test, test_size=0.5, random_state=42)

In [ ]:
from scipy.stats import entropy

js_divergences = np.zeros(x_train.shape[1])
for i in range(x_train.shape[1]):
    # Compute histograms with the same bins for both X_train and X_test
    bins = max(len(np.unique(x_train[:, i])), len(np.unique(x_test[:, i])))
    p, _ = np.histogram(x_train[:, i], bins=bins, density=True)
    q, _ = np.histogram(x_test[:, i], bins=bins, density=True)
    m = 0.5 * (p + q)
    js_divergences[i] = 0.5 * (entropy(p, m) + entropy(q, m))

avg_js_divergence = np.mean(js_divergences)
print("Average Jensen-Shannon divergence:", avg_js_divergence)

Average Jensen-Shannon divergence: 0.09482576054076818


### main

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(padded_x, padded_y, test_size=(1 - 0.7), random_state=42)
x_test, x_val, y_test, y_val = train_test_split(x_test, y_test, test_size=0.5, random_state=42)

#### FCN embedded

In [ ]:
class LSTMModel(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim):
        super(LSTMModel, self).__init__()
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.lstm1 = tf.keras.layers.LSTM(hidden_dim, return_sequences=True)
        self.dropout = tf.keras.layers.Dropout(0.5)  # Example dropout rate
        self.lstm2 = tf.keras.layers.LSTM(hidden_dim)
        self.fc = tf.keras.layers.Dense(output_dim, activation='softmax')

    def call(self, inputs):
        embedded = self.embedding(inputs)
        lstm_out1 = self.lstm1(embedded)
        lstm_out2 = self.lstm2(self.dropout(lstm_out1))
        output = self.fc(lstm_out2)
        return output

In [ ]:
vocab_size = len(tokenizer.word_index) + 1
embedding_dim = 100
hidden_dim = 128
output_dim = 25

In [ ]:
model = LSTMModel(vocab_size, embedding_dim, hidden_dim, output_dim)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(x_train, y_train, epochs=3, batch_size=20, validation_data=(x_val, y_val))

Epoch 1/3
516/516 [==============================] - 85s 145ms/step - loss: 1.7678 - accuracy: 0.3410 - val_loss: 0.8555 - val_accuracy: 0.6718
Epoch 2/3
516/516 [==============================] - 74s 143ms/step - loss: 0.5874 - accuracy: 0.7484 - val_loss: 0.2878 - val_accuracy: 0.8742
Epoch 3/3
516/516 [==============================] - 71s 138ms/step - loss: 0.1523 - accuracy: 0.9523 - val_loss: 0.0161 - val_accuracy: 0.9995


#### predict

In [ ]:
predictions = model.predict(x_test)

13/13 [==============================] - 1s 50ms/step


In [ ]:
import numpy as np

class_predictions = np.argmax(predictions, axis=1)

In [ ]:
from sklearn.metrics import accuracy_score

accuracy = accuracy_score(y_test, class_predictions)
print("Accuracy:", accuracy)

Accuracy: 0.9
